
# Alpha Earth – **Zonas KML** + NDVI (GEE + leafmap)

Este notebook extiende *Alpha Earth Foundations* para trabajar con **zonas delimitadas por KML**:
- Carga de KML desde **Asset de Earth Engine** *o* desde **URL KML** (conversión a GeoJSON).
- Disolver/normalizar zonas por un campo clave (p. ej. `zone_id`).
- Cálculo de **NDVI** (Sentinel-2 SR) y **estadísticas zonales**.
- Visualización en `leafmap/MapLibre` y **exportación** a Drive (raster y tabla).

> Si ya cargaste el KML como Asset (en el Code Editor de GEE), usa el **ID del Asset**. 
> Si no, puedes usar un **KML público por URL**; el notebook lo convierte a GeoJSON y luego a `FeatureCollection` de EE.


In [1]:

# === Dependencias ===
import sys

def _pip(cmd):
    print(f"Running: {cmd}")
    get_ipython().system(f"{sys.executable} -m pip install --quiet --upgrade {cmd}")

for p in ["earthengine-api", "geemap>=0.32.0", "leafmap>=0.34.0", "fastkml", "requests"]:
    _pip(p)

print("✅ Dependencias listas.")


Running: earthengine-api
Running: geemap>=0.32.0



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\USUARIO\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Running: leafmap>=0.34.0



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\USUARIO\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Running: fastkml



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\USUARIO\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Running: requests



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\USUARIO\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


✅ Dependencias listas.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: C:\Users\USUARIO\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:

# === Autenticación GEE ===
import ee
try:
    ee.Initialize()
    print("✅ EE inicializado")
except Exception:
    ee.Authenticate()
    ee.Initialize()
    print("✅ EE autenticado e inicializado")


EEException: Cannot authenticate: Invalid request.

In [ ]:

# === Imports ===
import ee, json, os, io, requests
import leafmap, geemap
from shapely.geometry import shape
from fastkml import kml

YEAR = 2025
START = ee.Date.fromYMD(YEAR, 7, 1)
END   = ee.Date.fromYMD(YEAR, 7, 31)
CLOUDY_PCT = 20

# Configura UNA de estas dos opciones:
KML_ASSET_ID = ""  # e.g., "users/tu_usuario/alphaearth/zonas_cafe"
KML_URL = ""       # e.g., "https://tusitio.com/zonas.kml" (público)

# Campo opcional para disolver/agrupar zonas (si existe en tu KML)
ZONE_KEY = "zone_id"  # o cambia por el nombre real del atributo; si vacío, no disuelve.



## Cargar Zonas desde KML
**Modo A.** Asset de GEE (`ee.FeatureCollection(KML_ASSET_ID)`)  
**Modo B.** URL KML pública → convertir a GeoJSON → `geemap.geojson_to_ee(...)`


In [ ]:

def load_zones_from_asset(asset_id: str) -> ee.FeatureCollection:
    fc = ee.FeatureCollection(asset_id)
    # Sanity check
    _n = fc.size().getInfo()
    print(f"📦 Cargadas { _n } features desde Asset: {asset_id}")
    return fc

def load_zones_from_kml_url(kml_url: str) -> ee.FeatureCollection:
    print(f"⬇️ Descargando KML desde URL: {kml_url}")
    r = requests.get(kml_url, timeout=60)
    r.raise_for_status()
    data = r.content

    # Parse KML → GeoJSON (vía fastkml)
    k = kml.KML()
    k.from_string(data)
    def _extract_features(kobj):
        feats = []
        for d in kobj.features():
            if hasattr(d, 'features'):
                feats.extend(_extract_features(d))
            else:
                feats.append(d)
        return feats

    feats = _extract_features(k)
    geojson = {"type": "FeatureCollection", "features": []}
    for f in feats:
        geom = f.geometry
        props = dict(f.extended_data.elements[0].data) if getattr(getattr(f, 'extended_data', None), 'elements', None) else {}
        # Convert shapely geom to GeoJSON mapping
        gj = {
            "type": "Feature",
            "geometry": json.loads(leafmap.geopandas_to_geojson(leafmap.shp_to_gdf(geom))) if False else json.loads(leafmap.geometry_to_geojson(geom)),
            "properties": props
        }
        geojson["features"].append(gj)

    # Convertir a EE FeatureCollection
    fc = geemap.geojson_to_ee(geojson)
    _n = fc.size().getInfo()
    print(f"📦 KML→GeoJSON→EE: { _n } features.")
    return fc

# Cargador resiliente
if KML_ASSET_ID:
    ZONES_FC = load_zones_from_asset(KML_ASSET_ID)
elif KML_URL:
    ZONES_FC = load_zones_from_kml_url(KML_URL)
else:
    # Fallback: un polígono de demostración
    print("⚠️ No se proporcionó KML. Usando un polígono de ejemplo cerca de Medellín.")
    demo_poly = ee.Geometry.Polygon(
        [[[-75.536, 6.262], [-75.486, 6.262], [-75.486, 6.235], [-75.536, 6.235], [-75.536, 6.262]]]
    )
    ZONES_FC = ee.FeatureCollection([ee.Feature(demo_poly, {"zone_id": "DEMO"})])

print("Zonas cargadas.")


In [ ]:

# (Opcional) Disolver por clave
def dissolve_by_key(fc: ee.FeatureCollection, key: str) -> ee.FeatureCollection:
    if not key:
        return fc
    # Agrupa por clave y fusiona geometrías
    keys = fc.aggregate_array(key).distinct()
    def _dissolve_one(k):
        k = ee.String(k)
        geom = fc.filter(ee.Filter.eq(key, k)).geometry().dissolve(30)
        return ee.Feature(geom, {key: k})
    dissolved = ee.FeatureCollection(keys.map(_dissolve_one))
    print("🧩 Disolución por clave completada.")
    return dissolved

ZONES = dissolve_by_key(ZONES_FC, ZONE_KEY) if ZONE_KEY else ZONES_FC
print("ZONES size:", ZONES.size().getInfo())



## NDVI (Sentinel-2 SR) y Estadísticas por Zona


In [ ]:

# Enmascarado de nubes S2 SR
def mask_s2_sr(image):
    qa = image.select('QA60')
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).divide(10000)\
                .select(['B2','B3','B4','B8','B11','B12','QA60'])\
                .copyProperties(image, image.propertyNames())

# Colección S2 y NDVI
s2_sr = (ee.ImageCollection("COPERNICUS/S2_SR")
         .filterBounds(ZONES.geometry())
         .filterDate(START, END)
         .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', CLOUDY_PCT))
         .map(mask_s2_sr))

s2_median = s2_sr.median()
ndvi = s2_median.normalizedDifference(['B8','B4']).rename('NDVI')

# Estadísticas zonales
reducer = (ee.Reducer.mean()
           .combine(ee.Reducer.median(), sharedInputs=True)
           .combine(ee.Reducer.minMax(), sharedInputs=True)
           .combine(ee.Reducer.stdDev(), sharedInputs=True))

stats = ndvi.reduceRegions(
    collection=ZONES,
    reducer=reducer,
    scale=10
)

print("✅ Estadísticas calculadas. Ejemplo primer feature:")
print(stats.first().toDictionary().getInfo())



## Visualización (leafmap / MapLibre)


In [ ]:

ndvi_vis = {
    'min': -0.2,
    'max': 0.9,
    'palette': ['#a50026', '#f46d43', '#fee08b', '#a6d96a', '#1a9850']
}
true_color_vis = {'bands': ['B4','B3','B2'], 'min': 0.0, 'max': 0.3}

center = ZONES.geometry().centroid().coordinates().getInfo()
center_latlon = [center[1], center[0]]

m = leafmap.Map(center=center_latlon, zoom=12, tiles="CartoDB.Positron")
m.add_basemap("Esri.WorldImagery")
m.addLayer(s2_median, true_color_vis, "Sentinel-2 True Color")
m.addLayer(ndvi, ndvi_vis, "NDVI (Sentinel-2)")
m.addLayer(ZONES.style(**{"color": "black", "fillColor": "00000000", "width": 2}), {}, "ZONAS KML")
m.add_colorbar(colors=ndvi_vis['palette'], vmin=ndvi_vis['min'], vmax=ndvi_vis['max'], label="NDVI")
m



## Exportaciones (opcionales)
- **Raster NDVI** recortado a zonas.
- **Tabla CSV** con estadísticas por zona.


In [ ]:

# Exportar tabla de estadísticas a Drive
# Descomenta si quieres exportar
# task_stats = ee.batch.Export.table.toDrive(
#     collection=stats,
#     description=f"AlphaEarth_ZonalStats_NDVI_{YEAR}",
#     folder="AlphaEarth_Exports",
#     fileNamePrefix=f"zonalstats_ndvi_{YEAR}",
#     fileFormat='CSV'
# )
# task_stats.start()
# print("🚚 Export de tabla iniciado:", task_stats.id)

# Exportar NDVI clip a las zonas (como imagen única sobre bounding geom de ZONES)
# task_ndvi = ee.batch.Export.image.toDrive(
#     image=ndvi.clip(ZONES.geometry()),
#     description=f"AlphaEarth_NDVI_clip_{YEAR}",
#     folder="AlphaEarth_Exports",
#     fileNamePrefix=f"ndvi_clip_{YEAR}",
#     scale=10,
#     region=ZONES.geometry(),
#     maxPixels=1e13
# )
# task_ndvi.start()
# print("🚚 Export NDVI iniciado:", task_ndvi.id)



## (Opcional) Guardar Zonas editadas como KML/GeoJSON locales

Si editas zonas dentro de Python, puedes exportarlas a **GeoJSON** local y luego convertir a **KML** con `leafmap` para reutilizarlas fuera de GEE.


In [ ]:

# Descargar las zonas a GeoJSON local (muestra/centroides simplificados para ejemplo)
# Nota: Para descargar geometrías completas a local, es más robusto exportar desde EE a Drive como tabla (ee.batch.Export.table.toDrive)
# Aquí mostramos un ejemplo simple para centroids:
centroids = ZONES.map(lambda f: ee.Feature(f.geometry().centroid(), f.toDictionary()))

# Export simple a lista de dicts (no recomendado para muchas features)
# Para pocas zonas funciona como ejemplo.
features_list = centroids.toList(1000)
gj = {"type": "FeatureCollection", "features": []}
n = features_list.size().getInfo()
for i in range(n):
    f = ee.Feature(features_list.get(i)).toGeoJSONString().getInfo()
    gj["features"].append(json.loads(f))

with open("/mnt/data/AlphaEarth_zones_centroids.geojson", "w", encoding="utf-8") as fp:
    json.dump(gj, fp)
print("💾 Guardado:", "/mnt/data/AlphaEarth_zones_centroids.geojson")

# Convertir ese GeoJSON a KML (solo como demo rápida)
leafmap.convert_file("/mnt/data/AlphaEarth_zones_centroids.geojson",
                     "/mnt/data/AlphaEarth_zones_centroids.kml")
print("💾 Guardado:", "/mnt/data/AlphaEarth_zones_centroids.kml")
